In [1019]:
#!g1.1
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils import Config
import os

In [1020]:
#!g1.1
def show_images(pred, mask):
    if np.max(pred) <= 1:
        pred = pred * 255
    if np.max(mask) <= 1:
        mask = mask * 255
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.axis('off')
    ax1.imshow(pred, cmap='gray')
    ax2.axis('off')
    ax2.imshow(mask, cmap='gray')
    plt.show()

In [1021]:
#!g1.1
!ls configs

UnetPP-efnb7-1024-v2.cfg  UnetPP-efnb7-2048.cfg


In [1022]:
#!g1.1
val_folder = '/home/jupyter/mnt/datasets/tissue/data/test'
val_names = [x.split('.')[0] for x in os.listdir(val_folder) if 'mask' not in x]
val_paths = []
for name in val_names:
    image_path = os.path.join(val_folder, name + '.jpg')
    mask_path = os.path.join(val_folder, name + '_mask.jpg')
    if not os.path.exists(image_path):
        print(f'{image_path} does not exist')
        continue
    if not os.path.exists(mask_path):
        print(f'{mask_path} does not exist')
        continue
    val_paths.append([os.path.join(val_folder, name + '.jpg'), os.path.join(val_folder, name + '_mask.jpg')])
val_paths = np.asarray(test_paths)
print(test_paths[:2])

['/home/jupyter/mnt/datasets/tissue/data/test/18-02791B_2019-05-07 22_25_06-lv1-14165-17366-6446-4855.jpg', '/home/jupyter/mnt/datasets/tissue/data/test/18-01592A_2019-05-07 21_59_18-lv1-18492-12001-4720-3984.jpg']


In [1023]:
#!g1.1
from production import ProductionModel

In [1024]:
#!g1.1
cfg_1 = Config().load(path=os.path.join('configs', 'UnetPP-efnb7-2048.cfg'))
model_1 = ProductionModel(cfg_1, cut_size=2048)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /tmp/xdg_cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


In [1025]:
#!g1.1
cfg_2 = Config().load(path=os.path.join('configs', 'UnetPP-efnb7-1024-v2.cfg'))
model_2 = ProductionModel(cfg_2, cut_size=1024)

In [1026]:
#!g1.1
import ttach as tta
transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90(angles=[0, 90, 180, 270]),
    ]
)

In [1027]:
#!g1.1
public_paths = [x[0] for x in test_paths]
folder = '/home/jupyter/mnt/datasets/tissue/data/test'
test_paths = [os.path.join(folder, x) for x in os.listdir(folder) if 'mask' not in x]

In [1028]:
#!g1.1
def apply_transform(model, image, transforms):
    preds = []
    for transformer in transforms: 
        augmented_image = transformer.augment_image(torch.unsqueeze(torch.from_numpy(image), dim=0))
        output = model(augmented_image)
        model_output = torch.unsqueeze(output, dim=0)
        deaug_mask = transformer.deaugment_mask(model_output)
        preds.append(deaug_mask[0])
    return torch.mean(torch.cat(preds, dim=0), dim=0)

In [1029]:
#!g1.1
output_folder = 'submit'

for image_path in tqdm.tqdm(test_paths[:3]):
    mask_path = os.path.join(output_folder, image_path.split('/')[-1])
    print(mask_path)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.transpose(image, (2, 0, 1))
        
    preds1 = apply_transform(model_1, image, transforms)
    preds2 = apply_transform(model_2, image, transforms)
    preds = preds1 * 0.5 + preds2 * 0.5
    binary = torch.where(preds > 0.5, 1, 0).numpy()
    cv2.imwrite(mask_path, binary * 255)
   

  0%|          | 0/3 [00:00<?, ?it/s]

submit/18-02791B_2019-05-07 22_25_06-lv1-14165-17366-6446-4855.jpg


 33%|███▎      | 1/3 [02:11<04:22, 131.47s/it]

submit/18-01592A_2019-05-07 21_59_18-lv1-18492-12001-4720-3984.jpg


 67%|██████▋   | 2/3 [03:04<01:48, 108.04s/it]

submit/18-01913A_2019-05-07 22_11_13-lv1-23474-19944-5901-4637.jpg


100%|██████████| 3/3 [04:25<00:00, 88.50s/it] 


In [854]:
#!g1.1
